In [7]:
pip install requests pandas openpyxl schedule

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import requests
import pandas as pd

def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": "false"
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching data:", response.status_code)
        return None


In [9]:
def process_data(data):
    df = pd.DataFrame(data, columns=[
        "name", "symbol", "current_price", "market_cap",
        "total_volume", "price_change_percentage_24h"
    ])
    
    df.rename(columns={
        "name": "Cryptocurrency",
        "symbol": "Symbol",
        "current_price": "Price (USD)",
        "market_cap": "Market Cap",
        "total_volume": "24h Volume",
        "price_change_percentage_24h": "24h % Change"
    }, inplace=True)
    
    return df


In [10]:
def analyze_data(df):
    top_5 = df.nlargest(5, "Market Cap")[["Cryptocurrency", "Market Cap"]]
    avg_price = df["Price (USD)"].mean()
    highest_change = df.nlargest(1, "24h % Change")[["Cryptocurrency", "24h % Change"]]
    lowest_change = df.nsmallest(1, "24h % Change")[["Cryptocurrency", "24h % Change"]]

    return top_5, avg_price, highest_change, lowest_change


In [11]:
from datetime import datetime

def save_to_excel(df):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"Live_Crypto_Data_{timestamp}.xlsx"
    
    with pd.ExcelWriter(file_name, engine="openpyxl", mode="w") as writer:
        df.to_excel(writer, sheet_name="Crypto Prices", index=False)
        
    print("Data saved to Excel:", file_name)



In [ ]:
import schedule
import time

def job():
    print("\nFetching latest data...")
    data = fetch_crypto_data()
    if data:
        df = process_data(data)
        save_to_excel(df)
        
        # Perform analysis
        top_5, avg_price, highest_change, lowest_change = analyze_data(df)
        
        print("\n🔹 Top 5 Cryptos by Market Cap:\n", top_5)
        print("\n🔹 Average Price of Top 50 Cryptos: $", round(avg_price, 2))
        print("\n🔹 Highest 24h % Change:\n", highest_change)
        print("\n🔹 Lowest 24h % Change:\n", lowest_change)

# Schedule task to run every 5 minutes
schedule.every(5).minutes.do(job)

print("Live crypto tracking started... Press Ctrl+C to stop.\n")
job()  # Run first job immediately

while True:
    schedule.run_pending()
    time.sleep(1)


Live crypto tracking started... Press Ctrl+C to stop.


Fetching latest data...
Data saved to Excel: Live_Crypto_Data_20250212_122836.xlsx

🔹 Top 5 Cryptos by Market Cap:
   Cryptocurrency     Market Cap
0        Bitcoin  1899250454203
1       Ethereum   313808511186
2         Tether   141919527352
3            XRP   138628734445
4         Solana    94938582709

🔹 Average Price of Top 50 Cryptos: $ 4152.41

🔹 Highest 24h % Change:
    Cryptocurrency  24h % Change
38            Dai       0.01391

🔹 Lowest 24h % Change:
    Cryptocurrency  24h % Change
20       Litecoin       -9.6096

Fetching latest data...
Data saved to Excel: Live_Crypto_Data_20250212_122836.xlsx

🔹 Top 5 Cryptos by Market Cap:
   Cryptocurrency     Market Cap
0        Bitcoin  1899485425549
1       Ethereum   313660023328
2         Tether   141920559546
3            XRP   138609065275
4         Solana    94713604212

🔹 Average Price of Top 50 Cryptos: $ 4151.34

🔹 Highest 24h % Change:
    Cryptocurrency  24h % Chang